General QC that you could run yourself! These code snippets are adapted from [https://choishingwan.github.io/PRS-Tutorial/target/#35-standard-gwas-qc](https://choishingwan.github.io/PRS-Tutorial/target/#35-standard-gwas-qc)

This notebook is quick and dry annotations/comments. They are not very extensive and I would suggest referring to the git hub repo referred to above.

First QC is to filter for keeping common variants and variants that pass HWE

In [ ]:
system("./plink --bfile Pherandom.reduced_1000_Genomes --maf 0.01 --allow-no-sex --hwe 1e-6 --geno 0.01 --mind 0.01 --write-snplist --make-just-fam --out QC/Pherandom.reduced_1000_Genome.QC", intern=T)

running with a window size of 200kb, sliding across the genome with step size of 50 variants at a time, and filter out any SNPs with LD r<sup>2</sup>  higher than 0.25

In [ ]:
system("./plink --bfile Pherandom.reduced_1000_Genomes --keep QC/Pherandom.reduced_1000_Genome.QC.fam --extract QC/Pherandom.reduced_1000_Genome.QC.snplist --indep-pairwise 200 50 0.25 --out QC/Pherandom.reduced_1000_Genome.QC", intern=T)

Checking for heterozygotes

In [ ]:
system("./plink --bfile Pherandom.reduced_1000_Genomes --extract QC/Pherandom.reduced_1000_Genome.QC.prune.in --keep QC/Pherandom.reduced_1000_Genome.QC.fam --het --out QC/Pherandom.reduced_1000_Genome.QC", intern=T)

Usually, you would throw out any relatives that are 2nd degree or more (or adjust it later in a mixed regression model using something known as a kinship matrix).
In this example, I used 0.99 because Laramie kept everyone

In [ ]:
system("./plink --bfile Pherandom.reduced_1000_Genomes --extract QC/Pherandom.reduced_1000_Genome.QC.prune.in --rel-cutoff 0.99 --out QC/Pherandom.reduced_1000_Genome.QC", intern=T)

In [ ]:
system("./plink --bfile Pherandom.reduced_1000_Genomes --make-bed --keep QC/Pherandom.reduced_1000_Genome.QC.rel.id --out QC/Pherandom.reduced_1000_Genome.QC --extract QC/Pherandom.reduced_1000_Genome.QC.snplist", intern=T)

There's sometimes variants and markers that do not match. These were taken almost verbatim from the git repo referred to in the first line above

In [ ]:
bim <- read.table("QC/Pherandom.reduced_1000_Genome.QC.bim")
colnames(bim) <- c("CHR", "SNP", "CM", "BP", "B.A1", "B.A2")
# Read in QCed SNPs
qc <- read.table("QC/Pherandom.reduced_1000_Genome.QC.snplist", header = F, stringsAsFactors = F)
# Read in MDD/SCZ data
MDD <-
    read.table(gzfile("MDD_2019_logORpVal"),
               header = T,
               stringsAsFactors = F, 
               sep="\t")

## Strand flipping
info <- merge(x=bim, y=MDD, by.x="SNP", by.y="SNP")
# Filter QCed SNPs
info <- info[info$SNP %in% qc$V1,]
# Function for finding the complementary allele
complement <- function(x) {
    switch (
        x,
        "A" = "T",
        "C" = "G",
        "T" = "A",
        "G" = "C",
        return(NA)
    )
}

In [ ]:
info$A1 <- sapply(info$A2, complement)



In [ ]:
# Get SNPs that have the same alleles across base and target
info.match <- subset(info, A1 == B.A1 & A2 == B.A2)



In [ ]:
# Identify SNPs that are complementary between base and target
info$C.A1 <- sapply(as.character(info$B.A1), complement)
info$C.A2 <- sapply(as.character(info$B.A2), complement)

In [ ]:
info.complement <- subset(info, A1 == C.A1 & A2 == C.A2)

In [ ]:
# Update the complementary alleles in the bim file
# This allow us to match the allele in subsequent analysis
complement.snps <- bim$SNP %in% info.complement$SNP
bim[complement.snps,]$B.A1 <-
    sapply(as.character(bim[complement.snps,]$B.A1), complement)
bim[complement.snps,]$B.A2 <-
    sapply(as.character(bim[complement.snps,]$B.A2), complement)

info.recode <- subset(info, A1 == B.A2 & A2 == B.A1)

In [ ]:
# Update the recode SNPs
recode.snps <- bim$SNP %in% info.recode$SNP
tmp <- bim[recode.snps,]$B.A1
bim[recode.snps,]$B.A1 <- bim[recode.snps,]$B.A2
bim[recode.snps,]$B.A2 <- tmp

In [ ]:
# identify SNPs that need recoding & complement
info.crecode <- subset(info, A1 == C.A2 & A2 == C.A1)

In [ ]:
# Update the recode + strand flip SNPs
com.snps <- bim$SNP %in% info.crecode$SNP
tmp <- bim[com.snps,]$B.A1
bim[com.snps,]$B.A1 <- as.character(sapply(as.character(bim[com.snps,]$B.A2), complement))
bim[com.snps,]$B.A2 <- as.character(sapply(as.character(tmp), complement))

In [ ]:
# Output updated bim file
write.table(
    bim,
    "QC/Pherandom.reduced_1000_Genome.QC.adj.bim",
    quote = F,
    row.names = F,
    col.names = F,
    sep="\t"
)

In [ ]:
#output mismatches
mismatch <-
    bim$SNP[!(bim$SNP %in% info.match$SNP |
                  bim$SNP %in% info.complement$SNP | 
                  bim$SNP %in% info.recode$SNP |
                  bim$SNP %in% info.crecode$SNP)]

write.table(
    mismatch,
    "QC/Pherandom.reduced_1000_Genome.QC.mismatch",
    quote = F,
    row.names = F,
    col.names = F
)

Renaming files so that you can run the plink files later for calculation

In [ ]:
system("mv QC/Pherandom.reduced_1000_Genome.QC.bim QC/Pherandom.reduced_1000_Genome.QC.bim.bk", intern=T)
system("mv QC/Pherandom.reduced_1000_Genome.QC.adj.bim QC/Pherandom.reduced_1000_Genome.QC.bim", intern=T)

LD clumping

In [ ]:
system("./plink --bfile QC/Pherandom.reduced_1000_Genome.QC --clump-p1 1 --clump-r2 0.1 --clump-kb 250 --clump MDD_2019_logORpVal --clump-snp-field SNP --clump-field P --out QC/Pherandom.reduced_1000_Genome.QC", intern=T)

Obtaining the valid SNPs

In [ ]:
system("awk 'NR!=1{print $3}' QC/Pherandom.reduced_1000_Genome.QC.clumped >  QC/Pherandom.reduced_1000_Genome.QC.clumped.valid.snp", intern=T)

Computing PRS

In [ ]:
system("./plink --bfile  QC/Pherandom.reduced_1000_Genome.QC --score MDD_2019_logORpVal 1 2 3 header no-mean-imputation --q-score-range q.ranges.GWASsig_to_1 MDD_2019_pvalue_score --extract QC/Pherandom.reduced_1000_Genome.QC.clumped.valid.snp --allow-no-sex --out OUTCOME/MDD", intern=T)